# Data Acquisition and Cleaning
<br>
Predictive modelling is highly dependent on the integrity of the data. For my part for the data acquisition, I first obtained data on the Oscar from 1927 -2020 from a reliable source on Kaggle as a starting point. Next I obtained the most up-to-date and reliable from TMDB using the TMDB API and also other useful data from the OMDB API which can be accessed using the IMDB ID which can be retrieve from the TMDB API
<br>
The aim of our project or more specifically our problem defination is to predict the winners of The 93rd Academy Awards which will be held later this month on 26th April 2021.
<br>
The first phase of the project is the Data Acquistion which will be dividied in a couple of steps:
<ol>1. Obtain Oscar Data from Kaggle and to scrape the data and only focus on the big 5 award categories.</ol>
<ol>2. Using the data collected, retrieve data from TMDB API.</ol>
<ol>3. Using the IMDB ID retrieved from the TMDB API, to collect more data such as total nomiations, total awards won, imdb rating, rotten 

In [1]:
# Importing neccesary libraries 
from tmdbv3api import TMDb, Movie, Discover, Genre
from tmdbv3api import Account
from tmdbv3api import Authentication
import numpy as np
import random
import pandas as pd
from collections import defaultdict
import requests as rq
import re
import traceback
import datetime
from collections import Counter
import omdb

In [2]:
##TMDB details
##Account details for getting movie reccomendations
##Run this block only once during a session
tmdb = TMDb()
tmdb.api_key = '0921b0cce35c0b2ec8b874614d1d0b47'
tmdb.language = 'en'

USERNAME = "cybercat94"
PASSWORD = "Wave1994@"
auth = Authentication(username=USERNAME, password=PASSWORD)
account = Account()
details = account.details()

print("You are logged in as %s. Your account ID is %s." % (details.username, details.id))
print("This session expires at: %s" % auth.expires_at)

You are logged in as cybercat94. Your account ID is 10198725.
This session expires at: 2021-04-07 15:37:17 UTC


## Scrapping and cleaning oscar_award_data.csv
We will start by scrapping the dataset to meet our needs and requirements.

In [3]:
oscar_df = pd.read_csv('data/the_oscar_award.csv')
oscar_df

,year_film,year_ceremony,ceremony,category,name,film,winner
0,1927,1928,1,ACTOR,Richard Barthelmess,The Noose,False
1,1927,1928,1,ACTOR,Emil Jannings,The Last Command,True
2,1927,1928,1,ACTRESS,Louise Dresser,A Ship Comes In,False
3,1927,1928,1,ACTRESS,Janet Gaynor,7th Heaven,True
4,1927,1928,1,ACTRESS,Gloria Swanson,Sadie Thompson,False
...,...,...,...,...,...,...,...
10390,2019,2020,92,WRITING (Original Screenplay),"Screenplay by Bong Joon Ho, Han Jin Won; Story...",Parasite,True
10391,2019,2020,92,JEAN HERSHOLT HUMANITARIAN AWARD,Geena Davis,NaN,True
10392,2019,2020,92,HONORARY AWARD,David Lynch,NaN,True
10393,2019,2020,92,HONORARY AWARD,Wes Studi,NaN,True


Renaming columns for better readabality and querying data from the year 1960 to 2020.

In [4]:
oscar_df= oscar_df.rename({'category':'oscar_category', 'name':'nominee', 'winner':'oscar_win'}, axis=1)
oscar_df.drop(['ceremony'], axis=1, inplace=True)
oscar_df = oscar_df.query('year_film >=1960')

Next is to get the total counts of nominations for each film.

In [5]:
no_of_wins_df = oscar_df[oscar_df['oscar_win']==True].groupby('film').size().reset_index(name='no_oscar_wins')
no_of_wins_df


,film,no_oscar_wins
0,'Round Midnight,1
1,12 Years a Slave,3
2,1917,3
3,20 Feet from Stardom,1
4,2001: A Space Odyssey,1
...,...,...
836,Z,2
837,Zero Dark Thirty,1
838,Zootopia,1
839,Zorba the Greek,3


In [6]:
no_of_nomination_df = oscar_df.groupby('film').size().reset_index(name='no_oscar_nominations')
df = pd.merge(no_of_wins_df, no_of_nomination_df, on=['film'], how='right').fillna(0)
pd.options.display.float_format = '{:,.0f}'.format
df

,film,no_oscar_wins,no_oscar_nominations
0,'38',0,1
1,'Crocodile' Dundee,0,1
2,'Round Midnight,1,2
3,(A) Torzija [(A) Torsion],0,1
4,...And Justice for All,0,2
...,...,...,...
3135,"sex, lies, and videotape",0,1
3136,the accountant,1,1
3137,the end,0,1
3138,Ådalen '31,0,1


For this project we will be focusing on major awards only, the so called "big 5".
<br>Best Actor
<br>Best Actress
<br>Best Director
<br>Best Picture
<br>Best Adapted Screenplay/Best Original Screenplay

<br>As the title of these awards kept changing through the years, we will have to filter through the data.


In [7]:
oscar_df = oscar_df.replace(to_replace={'oscar_category' : {'ACTOR IN A LEADING ROLE': 'Best Actor',
                                                            'ACTOR': 'Best Actor',
                                                            'ACTRESS': 'Best Actress',
                                                            'ACTRESS IN A LEADING ROLE': 'Best Actress',
                                                            'DIRECTING' : 'Best Director',
                                                            'BEST PICTURE' : 'Best Picture',
                                                            'BEST MOTION PICTURE' : 'Best Picture',
                                                            'WRITING (Adapted Screenplay)' : 'Best Adapted Screenplay',
                                                            'WRITING (Screenplay Adapted from Other Material)' : 'Best Adapted Screenplay',
                                                            'WRITING (Screenplay Based on Material from Another Medium)' : 'Best Adapted Screenplay',
                                                            'WRITING (Screenplay--based on material from another medium)' : 'Best Adapted Screenplay',
                                                            'WRITING (Screenplay Based on Material Previously Produced or Published)' : 'Best Adapted Screenplay',
                                                            'WRITING (Original Screenplay)' : 'Best Original Screenplay',
                                                            'WRITING (Screenplay Written Directly for the Screen)' : 'Best Original Screenplay',
                                                            'WRITING (Story and Screenplay--based on factual material or material not previously published or produced)' : 'Best Original Screenplay',
                                                            'WRITING (Story and Screenplay--written directly for the screen)' : 'Best Original Screenplay',
                                                            'WRITING (Screenplay Written Directly for the Screen--based on factual material or on story material not previously published or produced)' : 'Best Original Screenplay',
                                                            'WRITING (Story and Screenplay--based on material not previously published or produced)' : 'Best Original Screenplay'},}, value=None)

oscar_df = oscar_df.query('oscar_category in ["Best Actor", "Best Actress", "Best Director", "Best Picture", "Best Adapted Screenplay","Best Original Screenplay"]')
oscar_df = pd.merge(oscar_df, df, on=['film'], how='left')
oscar_df = oscar_df.reset_index()
oscar_df.drop(['index'], axis=1, inplace=True)
oscar_df

,year_film,year_ceremony,oscar_category,nominee,film,oscar_win,no_oscar_wins,no_oscar_nominations
0,1960,1961,Best Actor,Trevor Howard,Sons and Lovers,False,1,7
1,1960,1961,Best Actor,Burt Lancaster,Elmer Gantry,True,3,5
2,1960,1961,Best Actor,Jack Lemmon,The Apartment,False,5,10
3,1960,1961,Best Actor,Laurence Olivier,The Entertainer,False,0,1
4,1960,1961,Best Actor,Spencer Tracy,Inherit the Wind,False,0,4
...,...,...,...,...,...,...,...,...
1835,2019,2020,Best Original Screenplay,Written by Rian Johnson,Knives Out,False,0,1
1836,2019,2020,Best Original Screenplay,Written by Noah Baumbach,Marriage Story,False,1,6
1837,2019,2020,Best Original Screenplay,Written by Sam Mendes & Krysty Wilson-Cairns,1917,False,3,10
1838,2019,2020,Best Original Screenplay,Written by Quentin Tarantino,Once upon a Time...in Hollywood,False,2,10


# TMDB API

## Get Movie Information from TMDB api
This function uses TMDB api(DISCOVER), to retrieve all the movies with the constraint year.
<br> It will retrieve basic details such as id, title, list of genre ids, vote average and lastly vote count.
<br> We will later retrieve more information using the getDetails function to get details like casts, crews and etc.

In [8]:
def get_movies_by_year(year):

    movie = Movie()

    # initialiZe empty dictionary
    _movie_dict = {"tmdb_id":[], 
                   "year":[], 
                   "title":[], 
                   "genre":[], 
                   "tmdb_vote_average":[], 
                   "tmdb_vote_count":[]};
    
    print(f"In {year}")
    
    for i in range(1,501):
        discover = Discover()
        movie = discover.discover_movies({
            'year': year,
            'page': i,
            'certification_country': 'US'
        })
        
        for m in movie:
            _movie_dict["tmdb_id"].append(m.id)
            _movie_dict["year"].append(year)
            _movie_dict["title"].append(m.title)
            _movie_dict["genre"].append(m.title)
            _movie_dict["tmdb_vote_average"].append(m.title)
            _movie_dict["tmdb_vote_count"].append(m.title)

        if i == int(discover.total_pages):
            break

    return _movie_dict

Function to translate the genre id

In [9]:
def translate_genre_id(g):
    for i in range(0,len(g)):
        if g[i] == 28:
            g[i] = 'Action'
        elif g[i] == 12:
            g[i] = 'Adventure'
        elif g[i] == 16:
            g[i] = 'Animation'
        elif g[i] == 35:
            g[i] = 'Comedy'
        elif g[i] == 80:
            g[i] = 'Crime'
        elif g[i] == 99:
            g[i] = 'Documentary'
        elif g[i] == 18:
            g[i] = 'Drama'
        elif g[i] == 10751:
            g[i] = 'Family'
        elif g[i] == 14:
            g[i] = 'Fantasy'
        elif g[i] == 36:
            g[i] = 'History'
        elif g[i] == 27:
            g[i] = 'Horror'
        elif g[i] == 10402:
            g[i] = 'Music'
        elif g[i] == 9648:
            g[i] = 'Mystery'
        elif g[i] == 10749:
            g[i] = 'Romance'
        elif g[i] == 878:
            g[i] = 'Science Fiction'
        elif g[i] == 10770:
            g[i] = 'TV Movie'
        elif g[i] == 53:
            g[i] = 'Thriller'
        elif g[i] == 10752:
            g[i] = 'War'
        elif g[i] == 37:
            g[i] = 'Western'
    return g

Function to retrieve movie details with movie name and year

In [155]:
def search_movie_by_name_year(details_dict):
    
    oscar_dict = {"tmdb_id":[], 
                  "year_film":[], 
                  "year_ceremony":[], 
                  "film":[], 
                  "oscar_category":[], 
                  "nominee":[],
                  "no_oscar_wins":[], 
                  "no_oscar_nominations":[], 
                  "oscar_win":[],
                  "genre_id":[], 
                  "tmdb_vote_average":[],
                  "tmdb_vote_count":[]};
    
    movie = Movie()
    for item in details_dict:
        s = movie.search(item["film"])
        size = len(s)
        for i in range(0, size):
            result = s[i]
            try:
                if (result.release_date[0:4] == str(item["year_film"]) or result.release_date[0:4] == str(item["year_film"]-1) or result.release_date[0:4] == str(item["year_film"]-2)):
                    #print("found movie")
                    oscar_dict["tmdb_id"].append(result.id)
                    oscar_dict["year_film"].append(item["year_film"])
                    oscar_dict["year_ceremony"].append(item["year_ceremony"])
                    oscar_dict["oscar_category"].append(item["oscar_category"])
                    oscar_dict["film"].append(item["film"])
                    oscar_dict["nominee"].append(item["nominee"])
                    oscar_dict["no_oscar_wins"].append(item["no_oscar_wins"])
                    oscar_dict["no_oscar_nominations"].append(item["no_oscar_nominations"])
                    oscar_dict["oscar_win"].append(item["oscar_win"])
                    oscar_dict["genre_id"].append(result.genre_ids)
                    oscar_dict["tmdb_vote_average"].append(result.vote_average)
                    oscar_dict["tmdb_vote_count"].append(result.vote_count)
                    break
            except NameError:
                print("no release date")
            except:
                print("Error can't find movie")

    return oscar_dict

In [156]:
details_dict = oscar_df.to_dict('r')
df = pd.DataFrame(search_movie_by_name_year(details_dict))

/Users/ryu/opt/anaconda3/lib/python3.8/site-packages/pandas/core/frame.py:1485: FutureWarning: Using short name for 'orient' is deprecated. Only the options: ('dict', list, 'series', 'split', 'records', 'index') will be used in a future version. Use one of the above to silence this warning.
  warnings.warn(


Error can't find movie
Error can't find movie


In [157]:
df

,tmdb_id,year_film,year_ceremony,film,oscar_category,nominee,no_oscar_wins,no_oscar_nominations,oscar_win,genre_id,tmdb_vote_average,tmdb_vote_count
0,53939,1960,1961,Sons and Lovers,Best Actor,Trevor Howard,1,7,False,[18],7,18
1,22013,1960,1961,Elmer Gantry,Best Actor,Burt Lancaster,3,5,True,[18],7,85
2,284,1960,1961,The Apartment,Best Actor,Jack Lemmon,5,10,False,"[35, 18, 10749]",8,1429
3,18929,1960,1961,The Entertainer,Best Actor,Laurence Olivier,0,1,False,"[18, 10402]",6,28
4,1908,1960,1961,Inherit the Wind,Best Actor,Spencer Tracy,0,4,False,"[18, 36]",8,252
...,...,...,...,...,...,...,...,...,...,...,...,...
1833,546554,2019,2020,Knives Out,Best Original Screenplay,Written by Rian Johnson,0,1,False,"[35, 80, 18, 9648]",8,7011
1834,492188,2019,2020,Marriage Story,Best Original Screenplay,Written by Noah Baumbach,1,6,False,[18],8,4745
1835,530915,2019,2020,1917,Best Original Screenplay,Written by Sam Mendes & Krysty Wilson-Cairns,3,10,False,"[10752, 18, 28, 53]",8,7950
1836,466272,2019,2020,Once upon a Time...in Hollywood,Best Original Screenplay,Written by Quentin Tarantino,2,10,False,"[35, 18, 53]",8,8462


Function to retrieve top 4 casts 

In [13]:
def get_top_4_casts(creds):
    
    cast = (creds["cast"])
    cast_list = []
    
    for i in range(0,4):
        try:
            cast_list.append(cast[i].name)
        except:
            print("OUT OF RANGE")
        
    return cast_list

Function to retrieve total popularity score of top 4 casts 

In [14]:
def get_top_4_casts_popularity_score(creds):
    
    cast = (creds["cast"])
    popularity_score = 0

    for i in range(0,4):
        try:
            popularity_score += cast[i].popularity
        except:
            print("OUT OF RANGE")
        
    return popularity_score

Function to get total casts score

In [15]:
def get_casts_popularity_score(creds):
    
    cast = (creds["cast"])
    popularity_score = 0

    for c in cast:
        try:
            popularity_score += c.popularity
        except:
            print("OUT OF RANGE")
        
    return popularity_score

Function to retrieve crew based on positions

In [16]:
def get_crew_position(creds, job_id):
    crew = (creds["crew"])
    producer_list = []
    screenplay_list = []
    for c in crew:
        try:
            if c.job == 'Director' and job_id == 0:
                director = c.name
            elif c.job == 'Producer'and job_id == 1:
                producer_list.append(c.name)
            elif c.job == 'Screenplay'and job_id == 2:
                screenplay_list.append(c.name)
        except:
             print("OUT OF RANGE")
    if job_id == 0:
        return director
    elif job_id == 1:
        return producer_list
    elif job_id == 2:
        return screenplay_list

Function to get crew score

In [17]:
def get_crew_score(creds):
    crew = (creds["crew"])
    popularity_score = 0
    for c in crew:
        try:
            popularity_score += c.popularity
        except:
            print("OUT OF RANGE")
        
    return popularity_score

Function to get more movie details

In [158]:
def get_movie_details(oscar_dict):
    
    movie = Movie()
    
    movie_detials_dict = {"tmdb_id":[],
                           "imdb_id":[],
                           "year_film":[],
                           "year_ceremony":[], 
                           "film":[], 
                           "oscar_category":[], 
                           "nominee":[],
                           "oscar_win":[],
                           "no_oscar_wins":[], 
                           "no_oscar_nominations":[], 
                           "genre_id":[],
                           "staring_casts":[],
                           "casts_popularity":[],
                           "director":[],
                           "producers":[],
                           "screenplay":[],
                           "crew_popularity":[],
                           "budget":[],
                           "revenue":[],
                           "runtime":[],
                           "tmdb_vote_average":[], 
                           "tmdb_vote_count":[]};
                        
    for item in oscar_dict:
        if item["tmdb_id"] != 0:
            m = movie.details(item["tmdb_id"])
            movie_detials_dict["tmdb_id"].append(item["tmdb_id"])
            movie_detials_dict["imdb_id"].append(m.imdb_id)
            movie_detials_dict["year_film"].append(item["year_film"])
            movie_detials_dict["year_ceremony"].append(item["year_ceremony"])
            movie_detials_dict["film"].append(item["film"])
            movie_detials_dict["oscar_category"].append(item["oscar_category"])
            movie_detials_dict["nominee"].append(item["nominee"])
            movie_detials_dict["oscar_win"].append(item["oscar_win"])
            movie_detials_dict["no_oscar_wins"].append(item["no_oscar_wins"])
            movie_detials_dict["no_oscar_nominations"].append(item["no_oscar_nominations"])
            movie_detials_dict["genre_id"].append(item["genre_id"])
            movie_detials_dict["staring_casts"].append(get_top_4_casts(m.casts))
            movie_detials_dict["casts_popularity"].append(get_casts_popularity_score(m.casts))
            movie_detials_dict["director"].append(get_crew_position(m.casts, 0))
            movie_detials_dict["producers"].append(get_crew_position(m.casts, 1))
            movie_detials_dict["screenplay"].append(get_crew_position(m.casts, 2))
            movie_detials_dict["crew_popularity"].append(get_crew_score(m.casts))
            movie_detials_dict["budget"].append(m.budget)
            movie_detials_dict["revenue"].append(m.revenue)
            movie_detials_dict["runtime"].append(m.runtime)
            movie_detials_dict["tmdb_vote_average"].append(item["tmdb_vote_average"])
            movie_detials_dict["tmdb_vote_count"].append(item["tmdb_vote_count"])
                          
    return movie_detials_dict


In [159]:
oscar_dict = df.to_dict('r')
final_df = pd.DataFrame(get_movie_details(oscar_dict))

/Users/ryu/opt/anaconda3/lib/python3.8/site-packages/pandas/core/frame.py:1485: FutureWarning: Using short name for 'orient' is deprecated. Only the options: ('dict', list, 'series', 'split', 'records', 'index') will be used in a future version. Use one of the above to silence this warning.
  warnings.warn(


OUT OF RANGE
OUT OF RANGE
OUT OF RANGE
OUT OF RANGE
OUT OF RANGE


In [160]:
final_df

,tmdb_id,imdb_id,year_film,year_ceremony,film,oscar_category,nominee,oscar_win,no_oscar_wins,no_oscar_nominations,...,casts_popularity,director,producers,screenplay,crew_popularity,budget,revenue,runtime,tmdb_vote_average,tmdb_vote_count
0,53939,tt0054326,1960,1961,Sons and Lovers,Best Actor,Trevor Howard,False,1,7,...,21,Jack Cardiff,[],[Gavin Lambert],6,0,0,103,7,18
1,22013,tt0053793,1960,1961,Elmer Gantry,Best Actor,Burt Lancaster,True,3,5,...,39,Richard Brooks,[Bernard Smith],[Richard Brooks],15,3000000,10400000,146,7,85
2,284,tt0053604,1960,1961,The Apartment,Best Actor,Jack Lemmon,False,5,10,...,42,Billy Wilder,[Billy Wilder],[],40,3000000,25000000,125,8,1429
3,18929,tt0053796,1960,1961,The Entertainer,Best Actor,Laurence Olivier,False,0,1,...,38,Tony Richardson,[Harry Saltzman],"[John Osborne, Nigel Kneale]",32,0,0,96,6,28
4,1908,tt0053946,1960,1961,Inherit the Wind,Best Actor,Spencer Tracy,False,0,4,...,73,Stanley Kramer,"[Stanley Kramer, Herman Shumlin]","[Nedrick Young, Harold Jacob Smith]",21,0,0,128,8,252
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1833,546554,tt8946378,2019,2020,Knives Out,Best Original Screenplay,Written by Rian Johnson,False,0,1,...,140,Rian Johnson,"[Rian Johnson, Ram Bergman]",[],70,40000000,309232797,131,8,7011
1834,492188,tt7653254,2019,2020,Marriage Story,Best Original Screenplay,Written by Noah Baumbach,False,1,6,...,143,Noah Baumbach,"[Noah Baumbach, David Heyman]",[],82,19000000,2300000,137,8,4745
1835,530915,tt8579674,2019,2020,1917,Best Original Screenplay,Written by Sam Mendes & Krysty Wilson-Cairns,False,3,10,...,96,Sam Mendes,"[Sam Mendes, Callum McDougall, Pippa Harris, J...","[Sam Mendes, Krysty Wilson-Cairns]",161,100000000,374733942,119,8,7950
1836,466272,tt7131622,2019,2020,Once upon a Time...in Hollywood,Best Original Screenplay,Written by Quentin Tarantino,False,2,10,...,299,Quentin Tarantino,"[Quentin Tarantino, David Heyman, Shannon McIn...",[Quentin Tarantino],226,95000000,374251247,162,8,8462


# OMDB API
Next, we would like to get more information on the movie using the IMDB ID that we obtained using the TMDB API.
<br>We want to get information like:
<ol>1. Metascore </ol>
<ol>2. Imdb score </ol>
<ol>3. Rotten score </ol>
<ol>4. other nominations and wins </ol>


In [120]:
omdb.set_default('apikey', '4a7f4869')

In [175]:
def get_omdb_details(oscar_dict):
    
    omdb_detials_dict = {"imdb_id":[],
                         "oscar_category":[],
                         "nominee":[],
                         "imdb_rating":[],
                         "imdb_votes":[],
                         "rotten_tomatoes_rating":[],
                         "metascore":[],
                         "total_wins":[],
                         "total_nominations":[]};
    for item in oscar_dict:
        try:
            c = omdb.imdbid(item['imdb_id'])
            words = (c['awards'].split())
            imdb_rating = int((c['imdb_rating']).replace(".",""))
            imdb_votes = int((c['imdb_votes']).replace(',',''))
            if len(c['ratings']) > 1:
                rotten_tomatoes_rating = int((c['ratings'][1]['value'])[0:2])
            else: 
                rotten_tomatoes_rating = "N/A"
            if c['metascore'] != "N/A":
                metascore = int(c['metascore'])
            else:
                metascore = c['metascore']
            for key in words:
                try:
                    if key == "wins":
                        total_wins = int(words[(words.index(key)-1)])
                    if key == "nominations.":
                        total_nominations = int(words[(words.index(key)-1)])
                except:
                    print("not found")
            omdb_detials_dict["imdb_id"].append(item['imdb_id'])
            omdb_detials_dict["oscar_category"].append(item['oscar_category'])
            omdb_detials_dict["nominee"].append(item['nominee'])
            omdb_detials_dict["imdb_rating"].append((imdb_rating))
            omdb_detials_dict["imdb_votes"].append((imdb_votes))
            omdb_detials_dict["rotten_tomatoes_rating"].append((rotten_tomatoes_rating))
            omdb_detials_dict["metascore"].append((metascore))
            omdb_detials_dict["total_wins"].append(int(total_wins))
            omdb_detials_dict["total_nominations"].append(int(total_nominations))
        except:
            print("error")
            print(c['title'])
    return omdb_detials_dict

In [176]:
imdb = oscar_details.to_dict('r')
omdb_df = pd.DataFrame(get_omdb_details(imdb))

/Users/ryu/opt/anaconda3/lib/python3.8/site-packages/pandas/core/frame.py:1485: FutureWarning: Using short name for 'orient' is deprecated. Only the options: ('dict', list, 'series', 'split', 'records', 'index') will be used in a future version. Use one of the above to silence this warning.
  warnings.warn(


In [177]:
omdb_df

,imdb_id,oscar_category,nominee,imdb_rating,imdb_votes,rotten_tomatoes_rating,metascore,total_wins,total_nominations
0,tt0054326,Best Actor,Trevor Howard,71,1408,67,N/A,7,16
1,tt0053793,Best Actor,Burt Lancaster,78,10534,94,N/A,8,13
2,tt0053604,Best Actor,Jack Lemmon,83,165873,93,94,19,8
3,tt0053796,Best Actor,Laurence Olivier,71,3022,77,70,19,4
4,tt0053946,Best Actor,Spencer Tracy,81,27478,92,75,3,7
...,...,...,...,...,...,...,...,...,...
1833,tt8946378,Best Original Screenplay,Written by Rian Johnson,79,471083,97,82,47,107
1834,tt7653254,Best Original Screenplay,Written by Noah Baumbach,79,253928,94,94,126,269
1835,tt8579674,Best Original Screenplay,Written by Sam Mendes & Krysty Wilson-Cairns,83,442036,89,78,127,198
1836,tt7131622,Best Original Screenplay,Written by Quentin Tarantino,76,563506,85,83,134,372


In [178]:
final_movie_df = pd.merge(final_df, omdb_df, how='left', on=["imdb_id", "oscar_category", "nominee"])
final_movie_df

,tmdb_id,imdb_id,year_film,year_ceremony,film,oscar_category,nominee,oscar_win,no_oscar_wins,no_oscar_nominations,...,revenue,runtime,tmdb_vote_average,tmdb_vote_count,imdb_rating,imdb_votes,rotten_tomatoes_rating,metascore,total_wins,total_nominations
0,53939,tt0054326,1960,1961,Sons and Lovers,Best Actor,Trevor Howard,False,1,7,...,0,103,7,18,71,1408,67,N/A,7,16
1,22013,tt0053793,1960,1961,Elmer Gantry,Best Actor,Burt Lancaster,True,3,5,...,10400000,146,7,85,78,10534,94,N/A,8,13
2,284,tt0053604,1960,1961,The Apartment,Best Actor,Jack Lemmon,False,5,10,...,25000000,125,8,1429,83,165873,93,94,19,8
3,18929,tt0053796,1960,1961,The Entertainer,Best Actor,Laurence Olivier,False,0,1,...,0,96,6,28,71,3022,77,70,19,4
4,1908,tt0053946,1960,1961,Inherit the Wind,Best Actor,Spencer Tracy,False,0,4,...,0,128,8,252,81,27478,92,75,3,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1833,546554,tt8946378,2019,2020,Knives Out,Best Original Screenplay,Written by Rian Johnson,False,0,1,...,309232797,131,8,7011,79,471083,97,82,47,107
1834,492188,tt7653254,2019,2020,Marriage Story,Best Original Screenplay,Written by Noah Baumbach,False,1,6,...,2300000,137,8,4745,79,253928,94,94,126,269
1835,530915,tt8579674,2019,2020,1917,Best Original Screenplay,Written by Sam Mendes & Krysty Wilson-Cairns,False,3,10,...,374733942,119,8,7950,83,442036,89,78,127,198
1836,466272,tt7131622,2019,2020,Once upon a Time...in Hollywood,Best Original Screenplay,Written by Quentin Tarantino,False,2,10,...,374251247,162,8,8462,76,563506,85,83,134,372


In [179]:
db_path='Final_Movie_Data''.csv'
oscar_details.to_csv(db_path)
print("Exported to local folder:",db_path)

Exported to local folder: Final_Movie_Data.csv


# References
https://www.kaggle.com/unanimad/the-oscar-award
<br>http://www.omdbapi.com/
<br>https://developers.themoviedb.org/3
<br>https://pypi.org/project/tmdbv3api/